In [80]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F

In [81]:
import dgl.function as fn

class SAGEConv(nn.Module):
    """Graph convolution module used by the GraphSAGE model.

    Parameters
    ----------
    in_feat : int
        Input feature size.
    out_feat : int
        Output feature size.
    """
    def __init__(self, in_feat, out_feat):
        super(SAGEConv, self).__init__()
        # A linear submodule for projecting the input and neighbor feature to the output.
        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h):
        """Forward computation

        Parameters
        ----------
        g : Graph
            The input graph.
        h : Tensor
            The input node feature.
        """
        with g.local_scope():
            g.ndata['h'] = h
            # update_all is a message passing API.
            g.update_all(message_func=fn.copy_u('h', 'm'), reduce_func=fn.mean('m', 'h_N'))
            h_N = g.ndata['h_N']
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

In [82]:
class Model(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(Model, self).__init__()
        self.conv1 = SAGEConv(in_feats, h_feats)
        self.conv2 = SAGEConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat)
        h = F.relu(h)
        h = self.conv2(g, h)
        return h

In [83]:
import dgl.data

dataset = dgl.data.CoraGraphDataset()
g = dataset[0]

def train(g, model):
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    all_logits = []
    best_val_acc = 0
    best_test_acc = 0

    features = g.ndata['feat']
    labels = g.ndata['label']
    train_mask = g.ndata['train_mask']
    val_mask = g.ndata['val_mask']
    test_mask = g.ndata['test_mask']
    for e in range(200):
        # Forward
        logits = model(g, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that we should only compute the losses of the nodes in the training set,
        # i.e. with train_mask 1.
        loss = F.cross_entropy(logits[train_mask], labels[train_mask])

        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        test_acc = (pred[test_mask] == labels[test_mask]).float().mean()

        # Save the best validation accuracy and the corresponding test accuracy.
        if best_val_acc < val_acc:
            best_val_acc = val_acc
            best_test_acc = test_acc

        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        all_logits.append(logits.detach())

        if e % 5 == 0:
            print('In epoch {}, loss: {:.3f}, val acc: {:.3f} (best {:.3f}), test acc: {:.3f} (best {:.3f})'.format(
                e, loss, val_acc, best_val_acc, test_acc, best_test_acc))

model = Model(g.ndata['feat'].shape[1], 16, dataset.num_classes)
train(g, model)

  NumNodes: 2708
  NumEdges: 10556
  NumFeats: 1433
  NumClasses: 7
  NumTrainingSamples: 140
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.
In epoch 0, loss: 1.947, val acc: 0.072 (best 0.072), test acc: 0.091 (best 0.091)
In epoch 5, loss: 1.873, val acc: 0.356 (best 0.356), test acc: 0.383 (best 0.383)
In epoch 10, loss: 1.722, val acc: 0.534 (best 0.534), test acc: 0.538 (best 0.538)
In epoch 15, loss: 1.490, val acc: 0.612 (best 0.612), test acc: 0.605 (best 0.605)
In epoch 20, loss: 1.190, val acc: 0.672 (best 0.672), test acc: 0.645 (best 0.645)
In epoch 25, loss: 0.865, val acc: 0.704 (best 0.704), test acc: 0.690 (best 0.690)
In epoch 30, loss: 0.569, val acc: 0.712 (best 0.712), test acc: 0.724 (best 0.700)
In epoch 35, loss: 0.344, val acc: 0.724 (best 0.724), test acc: 0.747 (best 0.747)
In epoch 40, loss: 0.200, val acc: 0.730 (best 0.730), test acc: 0.759 (best 0.759)
In epoch 45, loss: 0.117, val acc: 0.742 (best 0.742), test acc:

In [84]:
def u_mul_e_udf(edges):
    return {'m' : edges.src['h'] * edges.data['w']}

In [85]:
class WeightedSAGEConv(nn.Module):
    """Graph convolution module used by the GraphSAGE model with edge weights.

    Parameters
    ----------
    in_feat : int
        Input feature size.
    out_feat : int
        Output feature size.
    """
    def __init__(self, in_feat, out_feat):
        super(WeightedSAGEConv, self).__init__()
        # A linear submodule for projecting the input and neighbor feature to the output.
        self.linear = nn.Linear(in_feat * 2, out_feat)

    def forward(self, g, h, w):
        """Forward computation

        Parameters
        ----------
        g : Graph
            The input graph.
        h : Tensor
            The input node feature.
        w : Tensor
            The edge weight.
        """
        with g.local_scope():
            g.ndata['h'] = h
            g.edata['w'] = w
            g.update_all(message_func=u_mul_e_udf, reduce_func=fn.mean('m', 'h_N'))
            h_N = g.ndata['h_N']
            h_total = torch.cat([h, h_N], dim=1)
            return self.linear(h_total)

In [86]:
class Model(nn.Module):
    def __init__(self, in_feats, h_feats, num_classes):
        super(Model, self).__init__()
        self.conv1 = WeightedSAGEConv(in_feats, h_feats)
        self.conv2 = WeightedSAGEConv(h_feats, num_classes)

    def forward(self, g, in_feat):
        h = self.conv1(g, in_feat, torch.ones(g.num_edges(), 1).to(g.device))
        h = F.relu(h)
        h = self.conv2(g, h, torch.ones(g.num_edges(), 1).to(g.device))
        return h

model = Model(g.ndata['feat'].shape[1], 16, dataset.num_classes)
train(g, model)

In epoch 0, loss: 1.951, val acc: 0.072 (best 0.072), test acc: 0.091 (best 0.091)
In epoch 5, loss: 1.863, val acc: 0.212 (best 0.212), test acc: 0.253 (best 0.253)
In epoch 10, loss: 1.696, val acc: 0.394 (best 0.466), test acc: 0.391 (best 0.461)
In epoch 15, loss: 1.449, val acc: 0.472 (best 0.472), test acc: 0.450 (best 0.450)
In epoch 20, loss: 1.138, val acc: 0.590 (best 0.590), test acc: 0.565 (best 0.565)
In epoch 25, loss: 0.806, val acc: 0.678 (best 0.678), test acc: 0.672 (best 0.672)
In epoch 30, loss: 0.515, val acc: 0.734 (best 0.734), test acc: 0.725 (best 0.725)
In epoch 35, loss: 0.304, val acc: 0.764 (best 0.764), test acc: 0.747 (best 0.747)
In epoch 40, loss: 0.173, val acc: 0.764 (best 0.764), test acc: 0.746 (best 0.747)
In epoch 45, loss: 0.099, val acc: 0.768 (best 0.768), test acc: 0.747 (best 0.743)
In epoch 50, loss: 0.060, val acc: 0.772 (best 0.772), test acc: 0.750 (best 0.750)
In epoch 55, loss: 0.038, val acc: 0.764 (best 0.772), test acc: 0.756 (best 0

In [87]:
torch.ones(g.num_edges(), 1)

tensor([[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]])